In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import utm

vancouver_path = "./vancouvercrime.csv"
crime_mapping_path = "./crime_type_mapping.csv"
eventDataDenver = pd.read_csv('Vancouver_events.csv')
df = pd.read_csv(vancouver_path)[:100]
crime_code_df = pd.read_csv(crime_mapping_path)

In [ ]:
fact_df = pd.DataFrame(columns=['location_Key','crime_key', 'date_key','event_key', 'is_traffic','is_fatal','is_nighttime'])
location_df = pd.DataFrame(columns=['location_Key','city','location_name', 'neighborhood', 'latitude','longitude','crime_rate'])
crime_df= pd.DataFrame(columns=['crime_key','crime_category_name', 'crime_type_name','crime_details',
                                 'start_hour', 'start_minute',"start_second",
                                 'end_hour', 'end_minute',"end_second",
                                 'report_hour', 'report_minute',"report_second"])
event_df = pd.DataFrame(columns=['event_key','event_name','event_type','event_location','event_neighbourhood','event_city','event_start_date','event_end_date','event_location_size'])

In [ ]:
#Event Dimension
for idx_event, row_event in eventDataVancouver.iterrows():
    event_row=[idx_event+30000000]
    event_row += [row_event['NAME'],row_event['TYPE'],row_event['LOCATION'],row_event['NEIGHBOURHOOD'],'Denver',row_event['DATE_START'],row_event['DATE_END'],row_event['LOCATION-SIZE']]
    event_df.loc[len(event_df)] = event_row
event_df.head()

In [ ]:
crime_list = crime_code_df["Vancouver"].tolist()

In [ ]:
for i, row in df.iterrows():
    if not(2015<=row['YEAR']<=2019):
        continue
        
    index = len(fact_df)
    
    # Van id starts from 10^7, Den id starts from 10^6
    location_id = index + 10000000
    city = "Vancouver"
    location = row["HUNDRED_BLOCK"]
    neighborhood = row["NEIGHBOURHOOD"]
    lat,lon = utm.to_latlon(row['X'],row['Y'],10,'U')
    crime_rate = 0
    location_df.loc[index] = [location_id,city,location,neighborhood,lat,lon,crime_rate]
    
    
    crime_id = index + 20000000
    
    crime_type_map = crime_code_df.iloc[crime_list.index(row['TYPE'])]
    crime_category_name = crime_type_map["corr Denver category"]
    crime_type_name = crime_type_map["corr Denver type"]
    crime_detail = crime_type_map["corr Denver detail"]
    
    na = np.nan
    year = row['YEAR']
    month = row['MONTH']
    day = row['DAY']
    hour = row['HOUR']
    minute = row['MINUTE']
    
    crime_df.loc[index] = [crime_id,crime_category_name,crime_type_name,crime_detail,hour,minute,0,na,na,na,na,na,na]
    
    str_month = str(month) if month>9 else '0'+str(month)
    str_day = str(day) if day>9 else '0'+str(day)
    date_id = int(str(year)+str_month+str_day)
    
    
    #Event
    event_id = np.nan
    for idx_event, row_event in event_df.iterrows():
        #the crime is in the event period
        if date_id>=row_event['event_start_date'] and date_id<=row_event['event_end_date']:
            #the event is city size and there is no event assign to the crime
            if row_event['event_location_size'] == 'city':
                if np.isnan(event_id):
                    event_id = row_event['event_key']
            #the event is not city size
            else:
                #the neighborhood are same
                if neighborhood == row_event['event_neighbourhood']:
                    event_id = row_event['event_key']
    
    is_night = 0 if 7 <= hour <= 18 else 1
    
    fatal_keywords=['FATAL','DEAD','DEATH','KILL','HOMICIDE','MURDER']
    is_fatal = 0
    for keyword in fatal_keywords:
        for name in [crime_category_name,crime_type_name,crime_detail]:
            if keyword in name.upper():
                is_fatal = 1

    
    fact_df.loc[index] = [location_id,crime_id,date_id,event_id,crime_type_map["is_traffic"],crime_type_map['is_fatal'],is_night]

In [ ]:
# fact_df convert float to int
for col in fact_df.columns:
    if col !="event_key":
        fact_df[[col]]=fact_df[[col]].astype(int)

In [ ]:
crime_df.head()

In [ ]:
location_df.head()

In [ ]:
fact_df.head()